In [1]:
import pandas as pd
import backtesting as bt
import pandas_ta as ta
import numpy as np
from datetime import datetime
from IPython.display import clear_output
import breeze_client as bc
import yfinance as yf
import time
from datetime import time


In [94]:
#df =bc.read_csv('data/NIFTY/NIFTY_options_23500_27-06-2024_1minute_7d.csv')
df = pd.read_csv('F:/Python Projects/stock market/stock-market-with-python/trading_api/2024-06-14_NSE.csv')
df['datetime'] = df['datetime'].apply(lambda x : datetime.fromtimestamp(int(x) / 1000))
df.set_index('datetime',inplace=True)
r_df = df['ltp'].resample('30S').ohlc()
r_df['volume'] = df['qty'].resample('1S').sum()
r_df


,open,high,low,close,volume
datetime,,,,,
2024-06-14 11:43:30,95.50,95.60,94.55,95.30,NaN
2024-06-14 11:44:00,95.15,96.10,94.80,95.25,50.0
2024-06-14 11:44:30,95.15,95.50,94.85,95.35,50.0
2024-06-14 11:45:00,95.40,95.95,95.00,95.65,75.0
2024-06-14 11:45:30,95.20,95.45,94.00,94.00,50.0
...,...,...,...,...,...
2024-06-14 15:27:30,89.35,90.50,88.95,89.75,375.0
2024-06-14 15:28:00,89.85,90.05,87.85,87.90,350.0
2024-06-14 15:28:30,88.15,88.25,87.10,87.30,275.0


In [95]:
r_df.rename(columns={
    'high':'High',
    'low':'Low',
    'open':'Open',
    'close':'Close',
    'volume':'Volume'},inplace=True)

r_df

,Open,High,Low,Close,Volume
datetime,,,,,
2024-06-14 11:43:30,95.50,95.60,94.55,95.30,NaN
2024-06-14 11:44:00,95.15,96.10,94.80,95.25,50.0
2024-06-14 11:44:30,95.15,95.50,94.85,95.35,50.0
2024-06-14 11:45:00,95.40,95.95,95.00,95.65,75.0
2024-06-14 11:45:30,95.20,95.45,94.00,94.00,50.0
...,...,...,...,...,...
2024-06-14 15:27:30,89.35,90.50,88.95,89.75,375.0
2024-06-14 15:28:00,89.85,90.05,87.85,87.90,350.0
2024-06-14 15:28:30,88.15,88.25,87.10,87.30,275.0


In [131]:
r_df.dropna(inplace=True)
r_df

,Open,High,Low,Close,Volume
datetime,,,,,
2024-06-14 11:44:00,95.15,96.10,94.80,95.25,50.0
2024-06-14 11:44:30,95.15,95.50,94.85,95.35,50.0
2024-06-14 11:45:00,95.40,95.95,95.00,95.65,75.0
2024-06-14 11:45:30,95.20,95.45,94.00,94.00,50.0
2024-06-14 11:46:00,94.00,94.20,93.40,93.85,100.0
...,...,...,...,...,...
2024-06-14 15:27:30,89.35,90.50,88.95,89.75,375.0
2024-06-14 15:28:00,89.85,90.05,87.85,87.90,350.0
2024-06-14 15:28:30,88.15,88.25,87.10,87.30,275.0


In [126]:
type(r_df['Close']/r_df['Open'].to_numpy())

pandas.core.series.Series

In [137]:
def supertrend(data,length=10,multiplier=3):

        super = ta.supertrend(data.High.s,data.Low.s,data.Close.s,length,multiplier) 

        return super.to_numpy().T[0]

def change(data):
        abc = (data.Close.s)/(data.Low.s)
        print(abc)
        return abc








In [138]:



class SuperTrend(bt.Strategy):

    multiplier = 4
    period = 10
    initial_qty = 250
    qty = initial_qty
    

    
    def init(self):
        
        self.super = self.I(supertrend,self.data,self.period,self.multiplier)   
        self.super = self.I(change,self.data)  
        

    
    def next(self):

        current_time = self.data.index[-1].time()

        current_price = self.data.Close[-1]
        last_price = self.data.Close[-2]
        current_trend = self.super[-1]
        last_trend = self.super[-2]

        #buy signal
        if (last_trend > last_price) and (current_trend < current_price):
            self.position.close()
            self.buy()
                
        #sell signal
        if (last_trend < last_price) and (current_trend > current_price):
            self.position.close()
            self.sell()

    
        

                
        

        


            


        
    

In [139]:
eng = bt.Backtest(r_df[:5000],SuperTrend,cash=100000)

In [140]:
stats = eng.run()
stats

datetime
2024-06-14 11:44:00    1.004747
2024-06-14 11:44:30    1.005271
2024-06-14 11:45:00    1.006842
2024-06-14 11:45:30    1.000000
2024-06-14 11:46:00    1.004818
                         ...   
2024-06-14 15:27:30    1.008994
2024-06-14 15:28:00    1.000569
2024-06-14 15:28:30    1.002296
2024-06-14 15:29:00    1.017796
2024-06-14 15:29:30    1.011364
Freq: 30S, Length: 452, dtype: float64


Start                     2024-06-14 11:44:00
End                       2024-06-14 15:29:30
Duration                      0 days 03:45:30
Exposure Time [%]                         0.0
Equity Final [$]                     100000.0
Equity Peak [$]                      100000.0
Return [%]                                0.0
Buy & Hold Return [%]                -6.56168
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

In [141]:
eng.plot()

C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p6244', ...)

In [118]:
def optim_func(series):

    if series['# Trades'] > 2:
        return series['Win Rate [%]']
    
    


    else:
        return 0

opt_stats,heatmap = eng.optimize(
    period = range(2,30),
    multiplier = range(1,6),
    
    #period = range(3,50),
    
    maximize = optim_func,
    
    return_heatmap = True,
    max_tries = 1000

)
opt_stats

Start                     2024-06-14 11:44:00
End                       2024-06-14 15:29:30
Duration                      0 days 03:45:30
Exposure Time [%]                    98.00885
Equity Final [$]                     148548.8
Equity Peak [$]                      156033.2
Return [%]                            48.5488
Buy & Hold Return [%]                -6.56168
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -15.722423
Avg. Drawdown [%]                   -4.050424
Max. Drawdown Duration        0 days 00:41:30
Avg. Drawdown Duration        0 days 00:09:33
# Trades                                    5
Win Rate [%]                            100.0
Best Trade [%]                      18.666667
Worst Trade [%]                      2.244536
Avg. Trade [%]                    

In [119]:
eng.plot()

C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p5720', ...)

In [120]:
bt.lib.plot_heatmaps(heatmap,agg='mean')

GridPlot(id='p5798', ...)

In [ ]:
class VWAP(bt.Strategy):

    length = 7

    def __init__(self):
        self.vwap = ta.vwap(self.data.High,self.data.Low,self.data.Close,self.data.Volume)

    def next(self):
        pass

In [19]:
resampled_df['vwap'] = ta.vwap(resampled_df.High,resampled_df.Low,resampled_df.Close,resampled_df.Volume)
resampled_df

,Open,High,Low,Close,Volume,vwap
datetime,,,,,,
2024-06-14 11:43:45,95.50,95.60,95.50,95.60,275,95.566667
2024-06-14 11:43:46,95.30,95.45,95.30,95.45,50,95.541026
2024-06-14 11:43:47,95.30,95.30,95.25,95.25,300,95.409333
2024-06-14 11:43:48,95.30,95.30,95.20,95.20,75,95.390476
2024-06-14 11:43:49,95.25,95.25,95.25,95.25,100,95.372917
...,...,...,...,...,...,...
2024-06-14 15:29:55,88.20,88.20,88.20,88.20,250,88.087914
2024-06-14 15:29:56,89.00,89.00,89.00,89.00,50,88.087940
2024-06-14 15:29:57,88.20,88.20,88.20,88.20,450,88.087969
